In [19]:
# !pip install rdflib
#% !pip install morph-kgc

In [20]:
import morph_kgc
g = morph_kgc.materialize('../mappings/config.ini')

g.serialize(destination='resultado.nt', format='nt')
print("Archivo resultado.nt generado")

INFO | 2025-11-12 10:45:32,517 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2025-11-12 10:45:33,953 | 53 mapping rules retrieved.
INFO | 2025-11-12 10:45:34,002 | Mapping partition with 38 groups generated.
INFO | 2025-11-12 10:45:34,005 | Maximum number of rules within mapping group: 3.
INFO | 2025-11-12 10:45:34,006 | Mappings processed in 1.485 seconds.
INFO | 2025-11-12 10:45:35,329 | Number of triples generated in total: 9466.


Archivo resultado.nt generado


c:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\plugins\serializers\nt.py:41: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


In [21]:
from rdflib import Graph
g = Graph()

In [22]:
# g.parse("../ontology/ontology.ttl", format="turtle")
# g.parse("resultado.nt", format="nt")

In [23]:
PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#> 
PREFIX schema: <https://schema.org/> 
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#> 
PREFIX dbo:    <http://dbpedia.org/ontology/> 
PREFIX owl:    <http://www.w3.org/2002/07/owl#> 
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#> 
"""


print(f"Triples cargados: {len(g)}")

# Mostrar las primeras 10 tripletas
for i, (s, p, o) in enumerate(g):
    print(s, p, o)
    if i >= 25:
        break

Triples cargados: 0


In [24]:
print(f"Triples cargados: {len(g)}")

PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
"""

def qrun(q, limit=50):
    q = PREFIXES + q
    res = g.query(q)
    print(f"--- {len(res)} filas ---")
    for i,row in enumerate(res):
        print([str(x) if x is not None else None for x in row])
        if i >= limit-1: break

Triples cargados: 0


In [25]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:EducationalInstitution   BIND("EducationalInstitution"    AS ?type) } UNION
  { ?x a schema:Library      BIND("PublicLibrary" AS ?type) } UNION
  { ?x a schema:ParkGarden   BIND("ParkGarden"    AS ?type) } UNION
  { ?x a sc:SportsCenter BIND("SportsCenter"  AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 1 filas ---


In [26]:
# 🔎 5 ejemplos de cada subclase (solo nombre)
# - Lista 5 nombres por subclase (si falta schema:name, usa el localname del IRI)

for subtype,label in [
    ("sc:EducationalInstitution","EducationalInstitutions"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {subtype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### EducationalInstitutions
--- 0 filas ---

### PublicLibraries
--- 0 filas ---

### ParksGardens
--- 0 filas ---

### SportsCenters
--- 0 filas ---


In [27]:
# 🚇 Número de individuos de Metro, Bus y Tren (Cercanías)
# - Tabla con tipo de transporte y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a schema:SubwayStation        BIND("Subway"        AS ?type) } UNION
  { ?x a schema:BusStation           BIND("Bus"           AS ?type) } UNION
  { ?x a schema:TrainStation BIND("SuburbanTrain" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 1 filas ---


In [28]:
# 🚏 5 ejemplos de Metro, Bus y Tren (solo nombre)
# - Muestra 5 nombres por tipo de transporte

for ttype,label in [
    ("schema:SubwayStation","Subway"),
    ("schema:BusStation","Bus"),
    ("schema:TrainStation","SuburbanTrain")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?t a {ttype} .
  OPTIONAL {{ ?t schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?t), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Subway
--- 0 filas ---

### Bus
--- 0 filas ---

### SuburbanTrain
--- 0 filas ---


In [29]:
# 🗺️ Número de individuos de Municipio, Distrito y Barrio
# - Tabla con tipo territorial y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a dbo:Municipality  BIND("Municipality"  AS ?type) } UNION
  { ?x a dbo:District      BIND("District"      AS ?type) } UNION
  { ?x a dbo:Neighbourhood BIND("Neighbourhood" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 1 filas ---


In [30]:
# 🏷️ 5 ejemplos de Municipio, Distrito y Barrio (solo nombre)
# - Muestra 5 nombres por cada clase territorial

for ctype,label in [
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood"),
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {ctype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Municipality
--- 0 filas ---

### District
--- 0 filas ---

### Neighbourhood
--- 0 filas ---


In [31]:
# 🔁 Comprobar jerarquía Municipio→Distrito→Barrio
# - Lista pares Municipio-Distrito y Distrito-Barrio (nombres) para verificar la estructura

print("### Neigbourhood → District")
qrun("""
SELECT ?municipality ?district WHERE {
  ?m a dbo:Neighbourhood ; schema:name ?municipality ; sc:locatedInDistrict ?d .
  OPTIONAL { ?d schema:name ?district }
}
ORDER BY ?municipality ?district
""", limit=20)

print("\n### District → Municipality")
qrun("""
SELECT ?district ?neighbourhood WHERE {
  ?d a dbo:District ; schema:name ?district ; sc:locatedInMunicipality ?n .
  OPTIONAL { ?n schema:name ?neighbourhood }
}
ORDER BY ?district ?neighbourhood
""", limit=20)


### Neigbourhood → District
--- 0 filas ---

### District → Municipality
--- 0 filas ---


In [32]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)



### Universities → Neighbourhood
--- 0 filas ---

### PublicLibraries → Neighbourhood
--- 0 filas ---

### ParksGardens → Neighbourhood
--- 0 filas ---

### SportsCenters → Neighbourhood
--- 0 filas ---


In [33]:
# 🚈 Comprobar relaciones subclases de Facility → Metro/Bus/Tren
# - Para cada subclase: cuenta cuántas instalaciones tienen cada tipo de transporte cercano

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    
    print(f"\n### {label} con transporte cercano (conteos por tipo)")
    qrun(f"""
SELECT ?transportType (COUNT(DISTINCT ?f) AS ?n) WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?transportType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?transportType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?transportType)
  }}
}}
GROUP BY ?transportType
ORDER BY DESC(?n)
""", limit=10)

    print(f"### {label} → ejemplos (nombre facility, tipo transporte, nombre transporte)")
    qrun(f"""
SELECT ?facility ?tType ?tName WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?t schema:name ?tname }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?tname, REPLACE(STR(?t), '^.*/', '')) AS ?tName)
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?tType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?tType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?tType)
  }}
}}
LIMIT 5
""", limit=5)



### Universities con transporte cercano (conteos por tipo)
--- 1 filas ---
### Universities → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---

### PublicLibraries con transporte cercano (conteos por tipo)
--- 1 filas ---
### PublicLibraries → ejemplos (nombre facility, tipo transporte, nombre transporte)


--- 0 filas ---

### ParksGardens con transporte cercano (conteos por tipo)
--- 1 filas ---
### ParksGardens → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---

### SportsCenters con transporte cercano (conteos por tipo)
--- 1 filas ---
### SportsCenters → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---


In [34]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:Facility   BIND("University"    AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")

--- 1 filas ---


In [35]:
# --------------------------------------------------------------------
query_2 = """
SELECT (COUNT(?park) AS ?total)  # Cambiamos ?count a ?total para evitar colisiones con el método
WHERE {
?park a schema:ParkGarden ;
        sc:accessibility "Instalación parcialmente accesible para personas con movilidad reducida" .
}
"""

print("\n--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---")
try:
    results_2 = g.query(PREFIXES + query_2)
    
    # COUNT devuelve una única fila. Iteramos sobre ella y accedemos al valor.
    for row in results_2:

        count_value = row.total.value if hasattr(row.total, 'value') else row.total
        print(f"Total de Parques Parcialmente Accesibles: {count_value}")

        
except Exception as e:
    print(f"Error al ejecutar la Consulta 2: {e}")


--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---
Total de Parques Parcialmente Accesibles: 0


In [36]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:Facility","Facilities")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)


### Facilities → Neighbourhood
--- 0 filas ---
